<a href="https://colab.research.google.com/github/13izay/Classificationcomparison/blob/main/Rock_Paper_Scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
   %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras


##Import the zip file and extract the contents

In [ ]:
#URL_="https://drive.google.com/open?id=1tsFxSWLlmZbWAoKqvbs6DS9IHJcvsbS3"
#path_to_zip= keras.utils.get_file("R_P_S.zip", origin=URL_, extract=True, cache_dir="/content/", cache_subdir="game_data")
#PATH= os.path.join(os.path.dirname(path_to_zip), "R_P_S")

####Extraction of contents using !unzip

In [ ]:
!unzip -q "/content/drive/My Drive/R_P_S.zip"

## Defining a path for the data

In [ ]:
PATH= os.path.join("/content/R_P_S")

###Defining directory path for training and validation data

In [ ]:
train_dir= os.path.join(PATH,"train")
valid_dir= os.path.join(PATH,"valid")
test_dir= os.path.join(PATH,"test")

##Set up variables for preprocessing

In [ ]:
batch_size=25
Epochs=20
IMG_HEIGHT= 500
IMG_WIDTH= 500


##Data Preparation

### Setting the image parameters in the format we want to download 

In [ ]:
train_data_generator= keras.preprocessing.image.ImageDataGenerator(rescale=1./255, 
                                                                   zoom_range=0.5,
                                                                   height_shift_range=.15,
                                                                   width_shift_range=.15,
                                                                   horizontal_flip=True,
                                                                   rotation_range=45)
                                                                   
valid_data_generator= ImageDataGenerator(rescale=1./255, 
                                        zoom_range=0.5,
                                        height_shift_range=.15,
                                        width_shift_range=.15,
                                        horizontal_flip=True,
                                        rotation_range=45)

###Downloading images from directory 

In [ ]:
train_data= train_data_generator.flow_from_directory(batch_size=batch_size, 
                                                     directory= train_dir, 
                                                     shuffle=True, 
                                                     target_size=[IMG_HEIGHT,IMG_WIDTH])
valid_data= valid_data_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=valid_dir,
                                                     shuffle=True,
                                                     target_size=[IMG_HEIGHT,IMG_WIDTH])

##Visualize images 

In [ ]:
sample_train_images,_= next(train_data) #extracts one batch of training image, (here:without label)

In [ ]:
plt.imshow(sample_train_images[24])

In [ ]:
def plotImages(img_array):
  fig, axes= plt.subplots(1,5, figsize=(10,10))
  axes= axes.flatten()
  for img, ax in zip(img_array, axes):
    ax.imshow(img)
    ax.axis("off")
  plt.tight_layout()
  plt.show()


In [ ]:
plotImages(sample_train_images[:5])

##Create model

In [ ]:
model= Sequential([
  Conv2D(16, 3, strides=1, padding="same", activation= "relu", input_shape=(IMG_HEIGHT, IMG_WIDTH,3)),
  MaxPooling2D(),
  

  Conv2D(32,3, strides=1, padding="same", activation="relu"),
  MaxPooling2D(),
  Dropout(0.2),

  Conv2D(64,3, strides=1, padding="SAME", activation="relu"),
  MaxPooling2D(),
  

   Conv2D(128,3 , strides=1, padding="SAME", activation="relu"),
   MaxPooling2D(),
   Dropout(0.2),

    Flatten(),
    Dense(128, activation="relu"),
  
    Dense(512, activation="relu"),
 
    Dense(3)
])

In [ ]:
#model= Sequential([
 #   Conv2D(16, 3, padding='same', activation='relu', 
  #         input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
   # MaxPooling2D(),
   # Dropout(0.2),
   # Conv2D(32, 3, padding='same', activation='relu'),
   # MaxPooling2D(),
   # Conv2D(64, 3, padding='same', activation='relu'),
   # MaxPooling2D(),
   # Dropout(0.2),
   # Flatten(),
   # Dense(512, activation='relu'),
   # Dense(1)
#])

In [ ]:
model.summary()

In [ ]:
model.compile(keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
history= model.fit_generator(
    train_data,
    validation_data=valid_data,
    epochs=Epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(Epochs)

plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()